# Preliminaries

In [1]:
import pandas as pd  #importing all the important packages
import numpy as np
import matplotlib.pyplot as plt #currently not used
import seaborn as sns #currently not used
import pickle

Import the table containing nationaldex pokemon data up until generation 8.

In [2]:
pickle_in = open('pokemon.pickle','rb')
pokemon = pickle.load(pickle_in)
pokemon.columns = pokemon.columns.str.upper().str.replace('_', '') 
pokemon['TYPE2'] = pokemon['TYPE2'].replace('None', value=np.nan)
pokemon['TYPE2'].fillna(pokemon['TYPE1'], inplace=True)
pokemon = pokemon.set_index('NAME')
pokemon=pokemon.drop(['ID'],axis=1)

Import the table containing all moves data up until generation 8.

In [3]:
pickle_in = open('moves.pickle','rb')
moves = pickle.load(pickle_in)
moves

,TYPE,CATEGORY,PP,POWER,ACCURACY,FLAVOUR
MOVE,,,,,,
Aerial Ace,flying,physical,20,60,101,"The user confounds the foe with speed, then sl..."
Aqua Jet,water,physical,20,40,100,The user lunges at the foe at a speed that mak...
Aqua Tail,water,physical,10,90,90,The user attacks by swinging its tail as if it...
Arm Thrust,fighting,physical,20,15,100,The user looses a flurry of open-palmed arm th...
Assurance,dark,physical,10,50,100,If the foe has already taken some damage in th...
...,...,...,...,...,...,...
Will-o-wisp,fire,status,15,--,75,"The user shoots a sinister, bluish white flame..."
Wish,normal,status,10,--,101,A self-healing move. The user restores its own...
Withdraw,water,status,40,--,101,The user withdraws its body into its hard shel...


We can import functions from other .ipynb files using:

In [4]:
from ipynb.fs.defs.Functions import type_weakness,dmg,base_stats_at_lvl,phys_spec_ratio,get_moves

We take the example of two pokemon with no IVs,EVs, or natures attacking one another. The function takes in the two pokemon and the move being used as well as their levels. 

In [5]:
def battle(p1_name,p1_lvl,p2_name,p2_lvl,move,pokemon,moves,crit):
    a_d = phys_spec_ratio(p1_name,p1_lvl,p2_name,p2_lvl,pokemon)
    move = moves[moves.index == move]
    if move.CATEGORY.values == 'physical':
        ratio = a_d[0]
    elif move.CATEGORY.values == 'special':
        ratio = a_d[1]
    else:
        ratio = 0
    p1 = pokemon[pokemon.index == p1_name]
    p2 = pokemon[pokemon.index == p2_name]
    STAB = (move.TYPE.values == p1.TYPE1.values) or (move.TYPE.values == p1.TYPE2.values)
    other = 1
    damage = dmg(ratio, p1_lvl, crit, move['POWER'].values.astype(str).astype(int), move['TYPE'].values,STAB,p2['TYPE1'].values,p2['TYPE2'].values,other)
    p2_hp = base_stats_at_lvl(p2_name,p2_lvl,pokemon)[0]
    return [int(damage[0]/p2_hp*100),int(damage[1]/p2_hp*100)]

In [6]:
battle('Charizard',30,'Venusaur',30,'Aerial Ace',pokemon,moves,True)

[54, 64]

We create a function to test the effectiveness of a pokemon using a move against all other pokemon to see the average damage done. This is run using the base stats at level 50 similar to a competitive pokemon battle.

In [7]:
from tqdm import tnrange, notebook
def move_effectiveness(p_name,move,pokemon,moves):
    dmg = [0, 0]
    ko = [0, 0]
    for index, row in pokemon.iterrows():
        d = battle(p_name,50,index,50,move,pokemon,moves,False)
        if d[0] > 100:
            d[0] = 100
            ko[0] = ko[0] + 1
        if d[1] > 100:
            d[1] = 100
            ko[1] = ko[1] + 1
        dmg = [dmg[0]+d[0],dmg[1]+d[1]]
    dmg[0] = dmg[0]/len(pokemon)
    dmg[1] = dmg[1]/len(pokemon)
    return [move,round(dmg[0],1),round(dmg[1],1),ko[0],ko[1]]

In [8]:
move_effectiveness('Charizard','Slash',pokemon,get_moves('Charizard'.lower()))

['Slash', 27.5, 32.3, 7, 10]

Using this new function move effectiveness in combination with the list of moves a pokemon knows we can create a dataframe of the pokemon's moves and their effectiveness against all pokemon.

In [9]:
def pokemon_effectiveness(p_name,pokemon):
    name = []
    kolb = []
    koub = []
    lb = []
    ub = []
    acc = []
    typ = []
    cat = []
    pokemon_moves = get_moves(p_name.lower())
    #pokemon_moves = pokemon_moves[pokemon_moves['CATEGORY'] == 'physical' or pokemon_moves['CATEGORY'] == 'special')]
    from tqdm import tnrange, notebook
    for index, row in notebook.tqdm(pokemon_moves.iterrows(),desc='main loop',total=len(pokemon_moves)):
        try:
            temp = []
            temp = move_effectiveness(p_name,index,pokemon,pokemon_moves)
            name.append(temp[0])
            lb.append(temp[1])
            ub.append(temp[2])
            kolb.append(temp[3])
            koub.append(temp[4])
            acc.append(pokemon_moves[pokemon_moves.index==index].ACCURACY.values)
            typ.append(pokemon_moves[pokemon_moves.index==index].TYPE.values)
            cat.append(pokemon_moves[pokemon_moves.index==index].CATEGORY.values)
        except ValueError as err:
            print(index,err)
        except TypeError as err:
            print(index,err)
    df = pd.DataFrame({'MOVE':name})
    df['LOWER BOUND'] = lb
    df['UPPER BOUND'] = ub
    df['GUARANTEED OHKO'] = kolb
    df['OHKO RANGE'] = koub
    df['ACCURACY'] = acc
    df['ACCURACY'] = df['ACCURACY'].astype(int)
    df['CONSISTENCY'] = df['LOWER BOUND'].astype(float)*df['ACCURACY'].astype(float)/100
    df['CONSISTENCY'] = df['CONSISTENCY'].round(1)
    df['TYPE'] = typ
    df['CATEGORY'] = cat
    df = df.set_index('MOVE')
    df = df.sort_values(by='CONSISTENCY',ascending=False)
    return df

In [10]:
eff = pokemon_effectiveness('Charizard',pokemon)

ValueError: Length of values does not match length of index

In [ ]:
eff = eff.sort_values(by='CONSISTENCY',ascending=False)
eff

In [ ]:
eff.style.apply(lambda x: ["background: white" if i == 0 else "" for i, v in enumerate(x)], axis=1)

In [ ]:
print(battle('Tepig',15,'Patrat',12,'Tackle',pokemon,get_moves('Tepig'.lower()),False))
print(battle('Tepig',15,'Patrat',12,'Ember',pokemon,get_moves('Tepig'.lower()),False))
print(battle('Tepig',15,'Patrat',12,'Flame Charge',pokemon,get_moves('Tepig'.lower()),False))